In [1]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import seaborn as sns
import re
import random as rd
import tensorflow as tf
from tensorflow.keras.layers.experimental.preprocessing import TextVectorization
from nltk import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords, wordnet
from sklearn.preprocessing import OneHotEncoder
import matplotlib.pyplot as plt
import autokeras as ak
from sklearn.model_selection import KFold

import nltk
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\ddayv\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
def Tokenize(f):     ## Pre-processando a frase

    ## Colocando em minusculo
    ## Retirando a pontuaçao
    ## Retirando as StopWords
    
    f = f.lower().replace('\n', '').replace('-','').replace('#','').replace('.','').replace(',','').replace('!','').replace('r\n','').replace('  ','').replace('https','').replace('rt','').replace('rn','')
    token = RegexpTokenizer(r"\w+")
    f = token.tokenize(f)
    
    stop_words = set(stopwords.words('portuguese'))
    
    new_word = [word for word in f if not word in stop_words]
    
    return ' '.join(new_word)

def remove_user(frase):
    frase = re.sub('@\w+','',frase)
    frase = re.sub('{https}[^ ]+','',frase)
    frase = re.sub('https\w+','',frase)
    # re.sub('#\w+','',frase)
    return frase

def pre_X(frases):
    lista = []
    
    for frase in frases:
        lista.append(frase)
        
    return lista

def pre_Y(number):
    lista = []
    
    for numb in number:
        lista.append(numb)
    
    return lista

def set_array(frases):
    
    vocab = []
    palavras = []
    for frase in frases:
        
        text_array = remove_user(frase)
        text_array = Tokenize(text_array)
        text_array = text_array.split(' ')
        for i in range(len(text_array)):
            vocab.append(text_array[i])
        
    return vocab


## KANSOAN

In [291]:
df = pd.read_csv("data.txt", sep=',', header=None, names=['text','sentiment'])

df_remove = df[df['sentiment'] == '#Inveja'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Raiva'].index
df = df.drop(df_remove)

df_remove = df[df['sentiment'] == '#Ironia'].index
df = df.drop(df_remove)


def binario(termo):
    if termo == '#Feliz' or termo == '#Amor':
        return 1
    elif termo == '#Triste' or termo == '#Chateado':
        return 0       
    
df['sentiment'] = df['sentiment'].apply(binario)

In [292]:
df['text'] = df['text'].apply(remove_user)
df['text'] = df['text'].apply(Tokenize)

In [293]:
text_KANSOAN = df['text']
sentiment = np.asarray(df['sentiment'])

count_vect = CountVectorizer()
tfidf_transformer = TfidfTransformer()

X_train_KANSOAN = count_vect.fit_transform(text_KANSOAN)
X_train_KANSOAN = tfidf_transformer.fit_transform(X_train_KANSOAN) # Aplicando o TF-IDF
X_train_KANSOAN.shape

(9224, 17314)

## TWEETSENT

In [294]:
df_tweet= pd.read_csv('export_TweetSentBR.csv')
df_tweet

,id,id_twitter,text,sentiment
0,1343,863044774588272640,Que coisa linda! O Programa #encontro estava m...,1
1,1344,865583716088766467,"Por mais #Encontro com as Irmãs Galvão, adorei...",1
2,1345,865063232201011201,Mr. CATRA @OficialMrCatra lançando sua nova mú...,1
3,1346,864668391008763905,quem viu aquela lutadora modela barbuda tatuad...,0
4,1347,865572794016378882,Tô passada com esse cara.... quanta merda pode...,-1
...,...,...,...,...
11565,12908,864636619000877056,eu ja to aqui pronto pro #MasterChefBR mas ain...,-1
11566,12909,863581588713603072,MALUCO! Uma coisa que eu não tenho coragem é e...,-1
11567,12910,864831041349054464,#MaisVoce @ANAMARIABRAGA está linda @RedeGlobo,1
11568,12911,863042798575951872,"Que orgulho de ti, @sportrecife! #Encontro",1


In [295]:
df_remove = df_tweet[df_tweet['sentiment'] == '-']
df_tweet = df_tweet.drop(df_remove.index)
df_remove = df_tweet[df_tweet['sentiment'] == '0']
df_tweet = df_tweet.drop(df_remove.index)

df_tweet['text'] = df_tweet['text'].apply(remove_user)
df_tweet['text'] = df_tweet['text'].apply(Tokenize)

In [296]:
df_tweet['sentiment'] = df_tweet['sentiment'].apply(lambda x: int(x))

In [297]:
text_Tweetsent = df_tweet['text']
polarity_ = np.asarray(df_tweet['sentiment'])

In [298]:
X_text = count_vect.transform(text_Tweetsent)

X_test_TWEET = tfidf_transformer.transform(X_text) # Aplicando o TF-IDF
X_test_TWEET.shape

(8605, 17314)

## TASH

In [331]:
df1 = pd.read_csv("tash-pt.csv")
df1 = df1.dropna()
df1 = df1.drop(columns=['id_twitter'])


In [332]:
df1['text'] = df1['text'].apply(remove_user)
df1['text'] = df1['text'].apply(Tokenize)

In [333]:
df1_remove = df1.drop(df1[df1['sentiment'] == 0].index)
sentiment_ = np.asarray(df1_remove['sentiment'])
text_TASH = df1_remove['text']

X_text_TASH = count_vect.transform(text_TASH)

X_test_TASH_ = tfidf_transformer.transform(X_text_TASH) # Aplicando o TF-IDF
X_test_TASH_.shape

(1769, 17314)

## UNILEX

In [302]:
df1_ = pd.read_excel('./rotulação/TweetsPolitical01 OK OK.xlsx')
df2 = pd.read_excel('./rotulação/TweetsPolitical02 OK OK.xlsx')
df3 = pd.read_excel('./rotulação/TweetsPolitical03 OK OK.xlsx')
df4 = pd.read_excel('./rotulação/TweetsPolitical04 OK OK.xlsx')
df5 = pd.read_excel('./rotulação/TweetsPolitical05 OK OK.xlsx')
df6 = pd.read_excel('./rotulação/TweetsPolitical06 OK OK.xlsx')
df7 = pd.read_excel('./rotulação/TweetsPolitical07 OK OK.xlsx')
df8 = pd.read_excel('./rotulação/TweetsPolitical08 OK OK.xlsx')
df9 = pd.read_excel('./rotulação/TweetsPolitical09 OK OK.xlsx')
df10 = pd.read_excel('./rotulação/TweetsPolitical10 OK OK.xlsx')
df11 = pd.read_excel('./rotulação/TweetsPolitical11 OK OK.xlsx')
df12 = pd.read_excel('./rotulação/TweetsPolitical12 OK OK.xlsx')
df13 = pd.read_excel('./rotulação/TweetsPolitical13 OK OK.xlsx')
df14 = pd.read_excel('./rotulação/TweetsPolitical14 OK OK.xlsx')
df15 = pd.read_excel('./rotulação/TweetsPolitical15 OK OK.xlsx')
lista = [df1_,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15]

In [303]:
for df_ in lista:
    df_.drop(df_[df_['Polaridade']== 0.0].index, inplace=True)
    

In [304]:

df_all = pd.concat(lista)

In [305]:
df_all = df_all.reset_index()

In [306]:
df_all

,index,Tweet,Polaridade
0,1,"- Bola pra frente que amanhã é outro dia, outr...",1.0
1,2,- Cara de mal? Acho que não... Apenas um corte...,1.0
2,3,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
3,4,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0
4,5,"""As mais #belas #descobertas ocorrem quando as...",1.0
...,...,...,...
7912,78,Bom diaa #SD Moisés Sou Boina Preta e Não Tem...,1.0
7913,79,"Sei que leve por divisa esse ""V"" que simboliza...",1.0
7914,80,L'ambiance d'hier au stade c'est grace ao Maca...,1.0
7915,81,Mais tarde tem mais SD no Chopp E Cia...hoje é...,1.0


In [307]:
df_all = df_all.dropna()

In [308]:
df_all

,index,Tweet,Polaridade
0,1,"- Bola pra frente que amanhã é outro dia, outr...",1.0
1,2,- Cara de mal? Acho que não... Apenas um corte...,1.0
2,3,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
3,4,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0
4,5,"""As mais #belas #descobertas ocorrem quando as...",1.0
...,...,...,...
7912,78,Bom diaa #SD Moisés Sou Boina Preta e Não Tem...,1.0
7913,79,"Sei que leve por divisa esse ""V"" que simboliza...",1.0
7914,80,L'ambiance d'hier au stade c'est grace ao Maca...,1.0
7915,81,Mais tarde tem mais SD no Chopp E Cia...hoje é...,1.0


In [309]:
df_all['Polaridade'].value_counts()

-1.0     4197
 1.0     3715
 10.0       1
 11.0       1
-2.0        1
Name: Polaridade, dtype: int64

In [310]:
remove1 = df_all[df_all['Polaridade'] == 11].index
df_all = df_all.drop(remove1).reset_index()
df_all = df_all.drop(columns=['index','level_0'])



In [311]:
remove2 = df_all[df_all['Polaridade'] == -2].index
df_all = df_all.drop(remove2).reset_index()


In [312]:
df_all = df_all.drop(columns=['index'])

In [313]:
remove3 = df_all[df_all['Polaridade'] == 10].index

In [314]:
df_all = df_all.drop(remove3).reset_index()
df_all = df_all.drop(columns=['index'])


In [315]:
df_all

,Tweet,Polaridade
0,"- Bola pra frente que amanhã é outro dia, outr...",1.0
1,- Cara de mal? Acho que não... Apenas um corte...,1.0
2,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
3,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0
4,"""As mais #belas #descobertas ocorrem quando as...",1.0
...,...,...
7907,Bom diaa #SD Moisés Sou Boina Preta e Não Tem...,1.0
7908,"Sei que leve por divisa esse ""V"" que simboliza...",1.0
7909,L'ambiance d'hier au stade c'est grace ao Maca...,1.0
7910,Mais tarde tem mais SD no Chopp E Cia...hoje é...,1.0


In [316]:
df_all['Polaridade'].value_counts()

-1.0    4197
 1.0    3715
Name: Polaridade, dtype: int64

In [317]:
df_all

,Tweet,Polaridade
0,"- Bola pra frente que amanhã é outro dia, outr...",1.0
1,- Cara de mal? Acho que não... Apenas um corte...,1.0
2,""" #PCdoB O PARTIDO QUE FAZ A DIFERENÇA: LEAL E...",1.0
3,"""@camaradamae: #PCdoB o Partido da coragem ! ...",1.0
4,"""As mais #belas #descobertas ocorrem quando as...",1.0
...,...,...
7907,Bom diaa #SD Moisés Sou Boina Preta e Não Tem...,1.0
7908,"Sei que leve por divisa esse ""V"" que simboliza...",1.0
7909,L'ambiance d'hier au stade c'est grace ao Maca...,1.0
7910,Mais tarde tem mais SD no Chopp E Cia...hoje é...,1.0


In [318]:
df_all['Tweet'] = df_all['Tweet'].apply(remove_user)
df_all['Tweet'] = df_all['Tweet'].apply(Tokenize)
polarity = np.asarray(df_all['Polaridade'])
Tweet = df_all['Tweet']


In [319]:
Tweet = count_vect.transform(Tweet)
X_test_UNILEX = tfidf_transformer.transform(Tweet) # Aplicando o TF-IDF
X_test_UNILEX.shape

(7912, 17314)

In [320]:
def func(num):
    if num == 0:
        return -1
    else:
        return 1

sentiment = df['sentiment'].apply(func)

## NB - TRAIN -> KANSOAN- TEST -> TWEET

In [21]:
## TIRAR OS SPLITS , até no treino

In [324]:
clf = MultinomialNB()
clf.fit(X_train_KANSOAN, sentiment)
y_pred = clf.predict(X_test_TWEET)

print(classification_report(polarity_,y_pred) )

              precision    recall  f1-score   support

          -1       0.49      0.90      0.63      3388
           1       0.85      0.38      0.53      5217

    accuracy                           0.59      8605
   macro avg       0.67      0.64      0.58      8605
weighted avg       0.71      0.59      0.57      8605



## NB - TRAIN -> KANSOAN - TEST -> UNILEX

In [325]:
clf = MultinomialNB()
clf.fit(X_train_KANSOAN, sentiment)
y_pred = clf.predict(X_test_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.60      0.87      0.71      4197
         1.0       0.70      0.36      0.47      3715

    accuracy                           0.63      7912
   macro avg       0.65      0.61      0.59      7912
weighted avg       0.65      0.63      0.60      7912



## NB - TRAIN -> KANSOAN - TEST -> TASH

In [334]:
clf = MultinomialNB()
clf.fit(X_train_KANSOAN, sentiment)
y_pred = clf.predict(X_test_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.57      0.84      0.68       881
           1       0.70      0.36      0.48       888

    accuracy                           0.60      1769
   macro avg       0.64      0.60      0.58      1769
weighted avg       0.64      0.60      0.58      1769



## SVM - TRAIN->KANSOAN - TEST-> TWEET

In [335]:
clf = SVC().fit(X_train_KANSOAN, sentiment) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(polarity_,y_pred))

              precision    recall  f1-score   support

          -1       0.52      0.80      0.63      3388
           1       0.80      0.52      0.63      5217

    accuracy                           0.63      8605
   macro avg       0.66      0.66      0.63      8605
weighted avg       0.69      0.63      0.63      8605



## SVM T KANSOAN T UNILEX

In [336]:
clf = SVC().fit(X_train_KANSOAN, sentiment) 
y_pred = clf.predict(X_test_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.61      0.66      0.63      4197
         1.0       0.58      0.53      0.55      3715

    accuracy                           0.60      7912
   macro avg       0.59      0.59      0.59      7912
weighted avg       0.59      0.60      0.59      7912



## SVM T KANSOAN T TASH

In [337]:
clf = SVC().fit(X_train_KANSOAN, sentiment) 
y_pred = clf.predict(X_test_TASH_)

print(classification_report(sentiment_,y_pred, zero_division=True))

              precision    recall  f1-score   support

          -1       0.58      0.69      0.63       881
           1       0.62      0.51      0.56       888

    accuracy                           0.60      1769
   macro avg       0.60      0.60      0.59      1769
weighted avg       0.60      0.60      0.59      1769



## RL T KANSOAN T TWEET

In [338]:
clf = LogisticRegression(max_iter=1000).fit(X_train_KANSOAN, sentiment) 
y_pred = clf.predict(X_test_TWEET)

print(classification_report(polarity_,y_pred))

              precision    recall  f1-score   support

          -1       0.54      0.77      0.63      3388
           1       0.79      0.57      0.66      5217

    accuracy                           0.65      8605
   macro avg       0.67      0.67      0.65      8605
weighted avg       0.69      0.65      0.65      8605



## RL T KANSOAN T UNILEX

In [339]:
clf = LogisticRegression(max_iter=1000).fit(X_train_KANSOAN, sentiment) 
y_pred = clf.predict(X_test_UNILEX)

print(classification_report(polarity,y_pred))

              precision    recall  f1-score   support

        -1.0       0.62      0.58      0.60      4197
         1.0       0.56      0.60      0.58      3715

    accuracy                           0.59      7912
   macro avg       0.59      0.59      0.59      7912
weighted avg       0.59      0.59      0.59      7912



## RL T KANSOAN T TASH


In [340]:
clf = LogisticRegression(max_iter=1000).fit(X_train_KANSOAN, sentiment) 
y_pred = clf.predict(X_test_TASH_)

print(classification_report(sentiment_,y_pred))

              precision    recall  f1-score   support

          -1       0.58      0.69      0.63       881
           1       0.62      0.51      0.56       888

    accuracy                           0.60      1769
   macro avg       0.60      0.60      0.60      1769
weighted avg       0.60      0.60      0.60      1769



In [341]:
one = OneHotEncoder(sparse=False)
teste_ = one.fit_transform(np.asarray(sentiment).reshape(-1,1))
nome = pd.DataFrame(teste_)
nome['coluna'] = 0
nome.rename(columns={1: 2, 'coluna':1} , inplace=True)
Y_KANSOAN = nome[[0,1,2]].to_numpy()

In [386]:
def func(num):
    if num == -1:
        return 0
    else:
        return 1

sentiment = df['sentiment']


In [351]:
polarity_ = np.asarray(df_tweet['sentiment'].apply(func))
polarity = np.asarray(df_all['Polaridade'].apply(func))
sentiment_ = np.asarray(df1_remove['sentiment'].apply(func))

In [385]:
np.unique(sentiment_)

array([0, 1], dtype=int64)

## TOTALMENT.C T KANSOAN T TWEET

In [401]:
model = tf.keras.Sequential([
    tf.keras.layers.Dense(50, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(25, activation='relu'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(10, activation='tanh'),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.Dense(1 , activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

y_one = np.asarray(sentiment)

fit = model.fit(X_train_KANSOAN.todense(), y_one, epochs=5, validation_data=(X_test_TWEET.todense(),polarity_))

Epoch 1/5
289/289 [==============================] - 3s 10ms/step - loss: 0.2927 - accuracy: 0.8914 - val_loss: 1.1529 - val_accuracy: 0.5748
Epoch 2/5
289/289 [==============================] - 2s 8ms/step - loss: 0.0296 - accuracy: 0.9928 - val_loss: 1.1588 - val_accuracy: 0.6353
Epoch 3/5
289/289 [==============================] - 2s 8ms/step - loss: 0.0076 - accuracy: 0.9990 - val_loss: 1.8518 - val_accuracy: 0.5650
Epoch 4/5
289/289 [==============================] - 2s 8ms/step - loss: 0.0043 - accuracy: 0.9993 - val_loss: 1.7714 - val_accuracy: 0.6030
Epoch 5/5
289/289 [==============================] - 2s 8ms/step - loss: 0.0037 - accuracy: 0.9997 - val_loss: 1.8181 - val_accuracy: 0.6157


In [402]:
predicted = model.predict(X_test_TWEET.todense())

def ajeita(predicted):
    for i in range(len(predicted)):
        if predicted[i,0] >= 0.5:
            predicted[i,0] = 1
        else:
            predicted[i,0] = 0
    return predicted

predicted = ajeita(predicted)
print(classification_report(polarity_, predicted))

              precision    recall  f1-score   support

           0       0.51      0.81      0.62      3388
           1       0.80      0.49      0.61      5217

    accuracy                           0.62      8605
   macro avg       0.65      0.65      0.62      8605
weighted avg       0.68      0.62      0.61      8605



## UNILEX

In [403]:
predicted = model.predict(X_test_UNILEX.todense())

predicted = ajeita(predicted)

print(classification_report(polarity, predicted))

              precision    recall  f1-score   support

           0       0.61      0.81      0.70      4197
           1       0.66      0.42      0.51      3715

    accuracy                           0.63      7912
   macro avg       0.64      0.61      0.61      7912
weighted avg       0.63      0.63      0.61      7912



## TASH

In [404]:
predicted = model.predict(X_test_TASH_.todense())

predicted = ajeita(predicted)

print(classification_report(sentiment_, predicted))

              precision    recall  f1-score   support

           0       0.59      0.70      0.64       881
           1       0.63      0.51      0.56       888

    accuracy                           0.60      1769
   macro avg       0.61      0.61      0.60      1769
weighted avg       0.61      0.60      0.60      1769



## LSTM T KANSOAN T UNILEX

In [405]:
vectorize_layer = TextVectorization(
 max_tokens=15000,
 output_mode='int',
 output_sequence_length=len(max(text_KANSOAN )))

vocab = set_array(text_KANSOAN )

vectorize_layer.adapt(np.unique(vocab))
len(vectorize_layer.get_vocabulary())


15000

In [406]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,mask_zero=True),
    
    tf.keras.layers.LSTM(50, activation='relu' ,return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(25 , activation='tanh', return_sequences=True),
    tf.keras.layers.Dropout(0.3),
    
    tf.keras.layers.LSTM(10 , activation='tanh'),
    tf.keras.layers.Dropout(0.2),
    
    tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

y_one = np.asarray(sentiment)


fit = model.fit(np.asarray(pre_X(text_KANSOAN)), y_one, epochs=5, batch_size=128,validation_data=(np.asarray(pre_X(df_all['Tweet'])),polarity))

Epoch 1/5
73/73 [==============================] - 13s 182ms/step - loss: 0.5994 - accuracy: 0.6573 - val_loss: 0.9047 - val_accuracy: 0.5616
Epoch 2/5
73/73 [==============================] - 11s 156ms/step - loss: 0.2269 - accuracy: 0.9412 - val_loss: 0.9556 - val_accuracy: 0.5828
Epoch 3/5
73/73 [==============================] - 12s 159ms/step - loss: 0.0804 - accuracy: 0.9838 - val_loss: 1.1096 - val_accuracy: 0.5933
Epoch 4/5
73/73 [==============================] - 12s 165ms/step - loss: 0.0414 - accuracy: 0.9931 - val_loss: 1.4133 - val_accuracy: 0.5748
Epoch 5/5
73/73 [==============================] - 12s 162ms/step - loss: 0.0285 - accuracy: 0.9952 - val_loss: 1.3914 - val_accuracy: 0.5948


In [407]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

predicted = ajeita(predicted)

print(classification_report(polarity,predicted))

              precision    recall  f1-score   support

           0       0.61      0.64      0.63      4197
           1       0.57      0.54      0.56      3715

    accuracy                           0.59      7912
   macro avg       0.59      0.59      0.59      7912
weighted avg       0.59      0.59      0.59      7912



## TWEET

In [408]:
predicted = model.predict(np.asarray(pre_X(text_Tweetsent)))

predicted = ajeita(predicted)

print(classification_report(polarity_, predicted))

              precision    recall  f1-score   support

           0       0.51      0.74      0.61      3388
           1       0.76      0.54      0.63      5217

    accuracy                           0.62      8605
   macro avg       0.64      0.64      0.62      8605
weighted avg       0.66      0.62      0.62      8605



## TASH

In [409]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

predicted = ajeita(predicted)

print(classification_report(sentiment_, predicted))

              precision    recall  f1-score   support

           0       0.58      0.61      0.60       881
           1       0.60      0.57      0.58       888

    accuracy                           0.59      1769
   macro avg       0.59      0.59      0.59      1769
weighted avg       0.59      0.59      0.59      1769



## CONV1D T KANSOAN T TASH

In [414]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Conv1D(32,6, activation='relu'),
    tf.keras.layers.MaxPooling1D(2),
    tf.keras.layers.Flatten(),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')   
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

y_one = np.asarray(sentiment)


fit = model.fit(np.asarray(pre_X(text_KANSOAN)), y_one, epochs=5, batch_size=128, validation_data=(np.asarray(pre_X(text_TASH)),sentiment_))

Epoch 1/5
73/73 [==============================] - 2s 28ms/step - loss: 0.5200 - accuracy: 0.7542 - val_loss: 0.9731 - val_accuracy: 0.5557
Epoch 2/5
73/73 [==============================] - 2s 23ms/step - loss: 0.0836 - accuracy: 0.9719 - val_loss: 1.2929 - val_accuracy: 0.5687
Epoch 3/5
73/73 [==============================] - 2s 26ms/step - loss: 0.0235 - accuracy: 0.9928 - val_loss: 1.2729 - val_accuracy: 0.5817
Epoch 4/5
73/73 [==============================] - 2s 26ms/step - loss: 0.0100 - accuracy: 0.9980 - val_loss: 1.3873 - val_accuracy: 0.5811
Epoch 5/5
73/73 [==============================] - 2s 29ms/step - loss: 0.0061 - accuracy: 0.9984 - val_loss: 1.5689 - val_accuracy: 0.5777


In [415]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

predicted = ajeita(predicted)

print(classification_report(sentiment_,predicted))

              precision    recall  f1-score   support

           0       0.62      0.38      0.47       881
           1       0.56      0.77      0.65       888

    accuracy                           0.58      1769
   macro avg       0.59      0.58      0.56      1769
weighted avg       0.59      0.58      0.56      1769



## TWEET

In [416]:
predicted = model.predict(np.asarray(pre_X(text_Tweetsent)))

predicted = ajeita(predicted)

print(classification_report(polarity_, predicted))

              precision    recall  f1-score   support

           0       0.59      0.52      0.55      3388
           1       0.71      0.76      0.73      5217

    accuracy                           0.67      8605
   macro avg       0.65      0.64      0.64      8605
weighted avg       0.66      0.67      0.66      8605



## UNILEX

In [417]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

predicted = ajeita(predicted)

print(classification_report(polarity, predicted))

              precision    recall  f1-score   support

           0       0.64      0.29      0.40      4197
           1       0.50      0.81      0.62      3715

    accuracy                           0.53      7912
   macro avg       0.57      0.55      0.51      7912
weighted avg       0.57      0.53      0.50      7912



## BDR T KANSOAN T TWEET

In [418]:
model = tf.keras.Sequential([
    vectorize_layer,
    tf.keras.layers.Embedding(
        input_dim=len(vectorize_layer.get_vocabulary()),
        output_dim=64,
        mask_zero=True),
    
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64)),
    
    tf.keras.layers.Dense(16, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
    
])

model.compile(
    optimizer= tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.binary_crossentropy,
    metrics=['accuracy']
)

y_one = np.asarray(sentiment)

fit = model.fit(np.asarray(pre_X(text_KANSOAN)), y_one, epochs=5, batch_size=128,validation_data=(np.asarray(pre_X(text_Tweetsent)),polarity_))

Epoch 1/5
73/73 [==============================] - 23s 311ms/step - loss: 0.3811 - accuracy: 0.8374 - val_loss: 0.8691 - val_accuracy: 0.6626
Epoch 2/5
73/73 [==============================] - 22s 307ms/step - loss: 0.0543 - accuracy: 0.9859 - val_loss: 0.9238 - val_accuracy: 0.6690
Epoch 3/5
73/73 [==============================] - 22s 301ms/step - loss: 0.0156 - accuracy: 0.9966 - val_loss: 1.1566 - val_accuracy: 0.6673
Epoch 4/5
73/73 [==============================] - 22s 307ms/step - loss: 0.0068 - accuracy: 0.9980 - val_loss: 1.4193 - val_accuracy: 0.6718
Epoch 5/5
73/73 [==============================] - 25s 341ms/step - loss: 0.0040 - accuracy: 0.9990 - val_loss: 1.5537 - val_accuracy: 0.6653


In [419]:
predicted = model.predict(np.asarray(pre_X(text_Tweetsent)))

predicted = ajeita(predicted)

print(classification_report(polarity_, predicted))

              precision    recall  f1-score   support

           0       0.57      0.59      0.58      3388
           1       0.73      0.72      0.72      5217

    accuracy                           0.67      8605
   macro avg       0.65      0.65      0.65      8605
weighted avg       0.67      0.67      0.67      8605



## UNILEX

In [420]:
predicted = model.predict(np.asarray(pre_X(df_all['Tweet'])))

predicted = ajeita(predicted)

print(classification_report(polarity,predicted))

              precision    recall  f1-score   support

           0       0.62      0.46      0.53      4197
           1       0.53      0.68      0.59      3715

    accuracy                           0.56      7912
   macro avg       0.57      0.57      0.56      7912
weighted avg       0.58      0.56      0.56      7912



## TASH

In [421]:
predicted = model.predict(np.asarray(pre_X(text_TASH)))

predicted = ajeita(predicted)

print(classification_report(sentiment_, predicted))

              precision    recall  f1-score   support

           0       0.63      0.47      0.54       881
           1       0.58      0.72      0.64       888

    accuracy                           0.60      1769
   macro avg       0.60      0.60      0.59      1769
weighted avg       0.60      0.60      0.59      1769

